In [1]:
import import_ipynb
from maze_common import Maze, shortestPathSearch
import time
import random as rnd
import numpy as np
import copy

importing Jupyter notebook from maze_common.ipynb


In [110]:
maze = Maze(15, .3)
solvable = maze.isSolvable()
print(maze.board, "\nIs solvable? ", solvable)

[[ 0  0  0  0 -1  0  0  0 -1  0 -1 -1 -1  0  0]
 [-1  0  0  0  0  0  0  0 -1  0  0  0 -1 -1  0]
 [ 0  0  0  0 -1  0  0  0  0 -1  0 -1  0  0  0]
 [-1 -1  0 -1  0 -1  0  0  0  0  0  0 -1  0  0]
 [ 0 -1  0  0  0 -1  0  0  0  0  0  0  0  0  0]
 [-1  0 -1  0 -1 -1  0 -1 -1  0  0  0  0  0 -1]
 [ 0  0  0  0  0  0  0 -1  0  0  0 -1  0  0  0]
 [ 0  0 -1  0 -1  0  0 -1 -1  0  0  0  0  0  0]
 [ 0 -1  0  0  0  0  0 -1  0  0 -1  0  0  0  0]
 [ 0 -1 -1  0  0  0  0 -1  0  0 -1  0 -1 -1  0]
 [ 0  0 -1 -1 -1 -1  0  0  0  0  0 -1  0 -1  0]
 [-1  0 -1  0 -1 -1  0  0  0 -1  0  0  0  0 -1]
 [-1 -1  0  0  0  0  0  0  0  0  0 -1  0  0  0]
 [ 0  0  0 -1  0  0  0 -1 -1 -1 -1 -1  0  0  0]
 [ 0  0 -1  0  0  0  0  0 -1  0  0  0 -1  0  0]] 
Is solvable?  True


In [27]:
def manhattanDistance(maze, cell):
    return abs(cell.row - (maze.dim - 1)) + abs(cell.col - (maze.dim - 1))

if (solvable):
    startTime = time.time()
    shortestPath = shortestPathSearch(maze, heuristicFunction = manhattanDistance)
    print("***** Found shortest path in %s seconds *****" % (time.time() - startTime))
    print(shortestPath)
    

***** Found shortest path in 0.2937893867492676 seconds *****
[(0, 0), (1, 0), (2, 0), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (8, 2), (8, 3), (9, 3), (10, 3), (10, 4), (10, 5), (11, 5), (11, 6), (12, 6), (12, 7), (13, 7), (14, 7), (14, 8), (14, 9), (14, 10), (14, 11), (14, 12), (14, 13), (14, 14)]


In [116]:
def thinMaze(maze, fractionRemove):
    thinMaze = Maze(1, 0)
    thinMaze.board = copy.deepcopy(maze.board)
    thinMaze.obstacles = copy.deepcopy(maze.obstacles)
    thinMaze.dim = maze.dim
    numRemove = round(fractionRemove * thinMaze.obstacles.shape[0])
    for i in range(0,numRemove):
        indexRemove = int(rnd.random() * thinMaze.obstacles.shape[0])
        obstacleX, obstacleY = thinMaze.obstacles[indexRemove]
        thinMaze.obstacles = np.delete(thinMaze.obstacles,indexRemove, axis=0)
        thinMaze.board[obstacleX, obstacleY] = 0
    return thinMaze
    
thinnedMaze = thinMaze(maze, .2)
print("Original obstacles: ", maze.obstacles.shape[0], "\n", maze.obstacles)
print("Thinned obstacles: ", thinnedMaze.obstacles.shape[0], "\n",thinnedMaze.obstacles)
print("Thinned maze:\n", thinnedMaze.board)

Original obstacles:  67 
 [[ 0  0]
 [ 0  4]
 [ 0  8]
 [ 0 10]
 [ 0 11]
 [ 0 12]
 [ 1  0]
 [ 1  8]
 [ 1 12]
 [ 1 13]
 [ 2  4]
 [ 2  9]
 [ 2 11]
 [ 3  0]
 [ 3  1]
 [ 3  3]
 [ 3  5]
 [ 3 12]
 [ 4  1]
 [ 4  5]
 [ 5  0]
 [ 5  2]
 [ 5  4]
 [ 5  5]
 [ 5  7]
 [ 5  8]
 [ 5 14]
 [ 6  7]
 [ 6 11]
 [ 7  2]
 [ 7  4]
 [ 7  7]
 [ 7  8]
 [ 8  1]
 [ 8  7]
 [ 8 10]
 [ 9  1]
 [ 9  2]
 [ 9  7]
 [ 9 10]
 [ 9 12]
 [ 9 13]
 [10  2]
 [10  3]
 [10  4]
 [10  5]
 [10 11]
 [10 13]
 [11  0]
 [11  2]
 [11  4]
 [11  5]
 [11  9]
 [11 14]
 [12  0]
 [12  1]
 [12 11]
 [13  3]
 [13  7]
 [13  8]
 [13  9]
 [13 10]
 [13 11]
 [14  2]
 [14  8]
 [14 12]
 [14 14]]
Thinned obstacles:  54 
 [[ 0  0]
 [ 0  8]
 [ 0 10]
 [ 0 11]
 [ 0 12]
 [ 1  8]
 [ 1 12]
 [ 1 13]
 [ 2  4]
 [ 2  9]
 [ 2 11]
 [ 3  0]
 [ 3  1]
 [ 3  3]
 [ 3  5]
 [ 3 12]
 [ 4  5]
 [ 5  0]
 [ 5  4]
 [ 5  5]
 [ 5  7]
 [ 5  8]
 [ 5 14]
 [ 6  7]
 [ 6 11]
 [ 7  2]
 [ 7  4]
 [ 7  8]
 [ 8  1]
 [ 8  7]
 [ 8 10]
 [ 9  2]
 [ 9 10]
 [ 9 13]
 [10  2]
 [10  3]
 [10  4]
 [10  5]
 [1

In [118]:
visited = {}

def thinnedMazeShortestPathLength(maze, cell):
    shortestPathLength = 0
    if ((cell.row, cell.col)) not in visited:
        shortestPath = shortestPathSearch(thinnedMaze, startCoords = (cell.row, cell.col))
        shortestPathLength = len(shortestPath)
        for i in range(0, shortestPathLength):
            row, col = shortestPath[i]
            if ((row, col)) in visited: 
                break
            else:
                visited[(row, col)] = shortestPathLength - i - 1
    else:
        shortestPathLength = visited[(cell.row, cell.col)]
    return shortestPathLength

if (solvable):
    startTime = time.time()
    shortestPath = shortestPathSearch(maze, heuristicFunction = thinnedMazeShortestPathLength)
    print("***** Found shortest path in %s seconds *****" % (time.time() - startTime))
    print(shortestPath)

***** Found shortest path in 14.558640956878662 seconds *****
[(0, 0), (0, 1), (0, 2), (0, 3), (1, 3), (1, 4), (1, 5), (1, 6), (2, 6), (2, 7), (2, 8), (3, 8), (3, 9), (4, 9), (5, 9), (6, 9), (7, 9), (8, 9), (9, 9), (10, 9), (10, 10), (11, 10), (11, 11), (11, 12), (12, 12), (12, 13), (12, 14), (13, 14), (14, 14)]


In [117]:
if (solvable):
    startTime = time.time()
    shortestPath = shortestPathSearch(thinnedMaze)
    print("***** Found shortest path in %s seconds *****" % (time.time() - startTime))
    print(shortestPath)

***** Found shortest path in 9.075846195220947 seconds *****
[(0, 0), (0, 1), (0, 2), (1, 2), (1, 3), (1, 4), (1, 5), (1, 6), (1, 7), (2, 7), (3, 7), (3, 8), (4, 8), (4, 9), (5, 9), (6, 9), (7, 9), (7, 10), (7, 11), (8, 11), (9, 11), (10, 11), (11, 11), (11, 12), (11, 13), (12, 13), (13, 13), (13, 14), (14, 14)]
